In [38]:
pip install requests pandas


In [39]:
import requests
import pandas as pd
import base64

# Swarm API details
login_url = 'https://bumblebee.hive.swarm.space/hive/login'
message_url = 'https://bumblebee.hive.swarm.space/hive/api/v1/messages'
username = 'INTECH'  # Replace with your username
password = 'InTechSWARM'  # Replace with your password
queryParams = '?status=2&count=1000'  # Adjust parameters as needed

# Login to Swarm
login_response = requests.post(login_url, data={'username': username, 'password': password})
cookie = login_response.headers['Set-Cookie']

# Fetch messages
full_message_url = message_url + queryParams
message_response = requests.get(full_message_url, headers={'Cookie': cookie})
messages = message_response.json()

# Decode and prepare data
data_rows = []
if messages:
    for message in messages:
        decoded_data = base64.b64decode(message['data']).decode('utf-8')
        data_variables = [item.zfill(4) for item in decoded_data.split(' ')]  # Pad and split data
        data_rows.append(data_variables)

# Convert to DataFrame
df = pd.DataFrame(data_rows)

# Define DataFrame columns
df.columns = ['Node', 'Date', 'Time', 'Battery Voltage', 'PV Input', 'Empty', 'Sleep Status']

# Correctly constructing the 'DateTime' string in 'YYYY-MM-DD HH:MM:SS' format
# This time ensuring the correct order of year, month, and day
df['DateTime'] = '20' + df['Date'].str[-2:] + '-' + df['Date'].str[:2] + '-' + df['Date'].str[2:4] + ' ' + df['Time'].str[:2] + ':' + df['Time'].str[2:] + ':00'

# Convert 'DateTime' strings to datetime objects
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')

# Check for any NaT values in 'DateTime' to ensure all conversions were successful
print(df[df['DateTime'].isna()])

# Set 'DateTime' as the index
df.set_index('DateTime', inplace=True)

# Drop the original 'Date' and 'Time' columns as they are now redundant
df.drop(columns=['Date', 'Time'], inplace=True)

# Assuming df is your DataFrame with 'DateTime' as the index and 'Battery Voltage' among the columns
df.sort_index(inplace=True)

# Display the DataFrame to confirm the changes
print(df.head())

# Convert 'Battery Voltage' to a numeric type (float)
df['Battery Voltage'] = pd.to_numeric(df['Battery Voltage'], errors='coerce')

# Check for any NaN values that result from conversion issues
print(df[df['Battery Voltage'].isna()])



ValueError: Length mismatch: Expected axis has 16 elements, new values have 7 elements

In [40]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Battery Voltage Over Time"),
    dcc.Graph(id="line-plot"),
    html.P("Select Node:"),
    dcc.Dropdown(
        id='node-dropdown',
        options=[{'label': node, 'value': node} for node in df['Node'].unique()],
        value=df['Node'].unique()[0],  # Default value
        clearable=False,
    ),
])

@app.callback(
    Output("line-plot", "figure"),
    [Input("node-dropdown", "value")]
)
def update_line_chart(selected_node):
    filtered_df = df[df['Node'] == selected_node]
    fig = px.line(filtered_df, y="Battery Voltage", title=f"Battery Voltage for Node {selected_node}")
    fig.update_xaxes(title_text='DateTime')
    fig.update_yaxes(title_text='Battery Voltage (V)')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


KeyError: 'Node'

In [ ]:
print(df.describe())


In [41]:
import pandas as pd
import base64

# Simulated base64-encoded messages in the format "Node, DateTime, Battery Voltage, PV Input, Sleep Timer"
encoded_messages = [
    base64.b64encode(b'N001,240318T2000Z,13.73,3,0'),
    base64.b64encode(b'N001,240318T2100Z,13.71,3,0'),
    base64.b64encode(b'N001,240318T2200Z,13.68,3,0'),
    base64.b64encode(b'N001,240319T2300Z,13.69,3,0'),
    base64.b64encode(b'N001,240319T0000Z,13.68,2,0'),
    # Add more messages as needed
]

# Decode messages and prepare data
data_rows = []
for msg in encoded_messages:
    decoded_data = base64.b64decode(msg).decode('utf-8')
    data_rows.append(decoded_data.split(','))

# Convert to DataFrame
df = pd.DataFrame(data_rows, columns=['Node', 'DateTime', 'Battery Voltage', 'PV Input', 'Sleep Timer'])

# Convert 'DateTime' strings to datetime objects using the compact format
df['DateTime'] = pd.to_datetime(df['DateTime'], format='%y%m%dT%H%MZ', errors='coerce')

# Set 'DateTime' as the index
df.set_index('DateTime', inplace=True)

# Convert 'Battery Voltage', 'PV Input', and 'Sleep Timer' to numeric types
df['Battery Voltage'] = pd.to_numeric(df['Battery Voltage'])
df['PV Input'] = pd.to_numeric(df['PV Input'])
df['Sleep Timer'] = pd.to_numeric(df['Sleep Timer'])

# Display the DataFrame to confirm the changes
print(df.head())


                     Node  Battery Voltage  PV Input  Sleep Timer
DateTime                                                         
2024-03-18 20:00:00  N001            13.73         3            0
2024-03-18 21:00:00  N001            13.71         3            0
2024-03-18 22:00:00  N001            13.68         3            0
2024-03-19 23:00:00  N001            13.69         3            0
2024-03-19 00:00:00  N001            13.68         2            0


In [42]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import plotly.express as px

# Assuming df is your DataFrame with 'DateTime' as the index and 'Battery Voltage' among the columns
df.sort_index(inplace=True)

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Battery Voltage Over Time"),
    dcc.Graph(id="line-plot"),
    html.P("Select Node:"),
    dcc.Dropdown(
        id='node-dropdown',
        options=[{'label': node, 'value': node} for node in df['Node'].unique()],
        value=df['Node'].unique()[0],  # Default value
        clearable=False,
    ),
])

@app.callback(
    Output("line-plot", "figure"),
    [Input("node-dropdown", "value")]
)
def update_line_chart(selected_node):
    filtered_df = df[df['Node'] == selected_node]
    fig = px.line(filtered_df, y="Battery Voltage", title=f"Battery Voltage for Node {selected_node}")
    fig.update_xaxes(title_text='DateTime')
    fig.update_yaxes(title_text='Battery Voltage (V)')
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)
